<a href="https://colab.research.google.com/github/jsja22/ai/blob/master/GHI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
import glob
import sys
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential,Model ,load_model
from tensorflow.keras.layers import Dense, LSTM, Dropout,Lambda,MaxPooling2D, Conv2D, Flatten, Reshape, Conv1D, MaxPooling1D, Input,LeakyReLU
from sklearn.metrics import mean_squared_error,r2_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
import math


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

'''
def preprocess_data(data,is_train=True):
    
    #시간각 구하기(w)   12시기준으로 12시 이후 1시간 단위로 15도씩 증가 12시 이전으로 1시간 단위로 -15도 씩 증가
    angle = 15
    noon = 12  
    data['time_angle'] = [(x - noon ) * angle if x >= noon  else -(noon  - x) * angle for x in data.Hour]

    #일적위 구하기 -23.45*np.cos(360/365 *(x+10))
    data['declination'] = [-23.44 * np.cos(360 /365 *(x+10)) for x in data.Day]
    
    #cos세타제트 구하기  => CosФCosбCosω + SinρSinФ
    data['theta_z'] = 90 - 1/(np.sin(np.sin(data['declination']) *np.sin(36) + \
    np.cos(data['declination'])*np.cos(36) * np.cos(data['time_angle'])))    

    #GHI 구하기  ->DHI + DNIx(cos세타제트)
    data['GHI'] = data.DHI + data.DNI * np.cos(data.theta_z)

    #data['Time'] = data['Hour'] + data['Minute']*(0.5/30)
    #data['sin_time'] = np.sin(2*np.pi*data.Time/24)
    #data['cos_time'] = np.cos(2*np.pi*data.Time/24)
    
    temp = data.copy()
    temp = temp[['DHI','DNI','GHI','T','WS','RH','TARGET']]
    

    if is_train == True:
        temp['TARGET1'] = temp['TARGET'].shift(-48).fillna(method = 'ffill')
        temp['TARGET2'] = temp['TARGET'].shift(-96).fillna(method = 'ffill')
        temp = temp.dropna()
        return temp.iloc[:-96]

    elif is_train == False:
        return temp.iloc[-48*day:, :]
'''
def preprocess_data(data,is_train=True):
    data['cos'] = np.cos(np.pi/2 - np.abs(data['Hour']%12 - 6)/6*np.pi/2)
    data.insert(1,'GHI',data['DNI']*data['cos']+data['DHI'])

    data['Time'] = data['Hour'] + data['Minute']*(0.5/30)
    data['sin_time'] = np.sin(2*np.pi*data.Time/24)
    data['cos_time'] = np.cos(2*np.pi*data.Time/24)
    temp = data.copy()
    temp = temp[['DHI','DNI','GHI','T','WS','RH','TARGET']]
    

    if is_train == True:
        temp['TARGET1'] = temp['TARGET'].shift(-48).fillna(method = 'ffill')
        temp['TARGET2'] = temp['TARGET'].shift(-96).fillna(method = 'ffill')
        temp = temp.dropna()
        return temp.iloc[:-96]

    elif is_train == False:
        return temp.iloc[-48*day:, :]




df_train = preprocess_data(train,is_train=True)
print(df_train.columns) 

Index(['DHI', 'DNI', 'GHI', 'T', 'WS', 'RH', 'TARGET', 'TARGET1', 'TARGET2'], dtype='object')


In [12]:
print(df_train['GHI'].iloc[8555:8900])  #8555    157.127313
#8556     48.000000
#8557     57.000000
#8558    263.255389
#8559    281.678703
           ...    
#8895    282.455541
#8896    489.000000
#8897    517.500000
#8898    681.042243
#8899    705.494516

#공식 대입 안했을때

8555    157.127313
8556     48.000000
8557     57.000000
8558    263.255389
8559    281.678703
           ...    
8895    282.455541
8896    489.000000
8897    517.500000
8898    681.042243
8899    705.494516
Name: GHI, Length: 345, dtype: float64


In [6]:
print(df_train['GHI'].iloc[8555:8900]) #8555    354.173473
#8556    463.123872
#8557    534.938669
#8558    573.653065
#8559    611.526886
           ...    
8895    785.866587
#8896    728.558497
#8897    713.864446
#8898    809.998912
#8899    821.154507

#ghi공식 대입 했을때

8555    354.173473
8556    463.123872
8557    534.938669
8558    573.653065
8559    611.526886
           ...    
8895    785.866587
8896    728.558497
8897    713.864446
8898    809.998912
8899    821.154507
Name: GHI, Length: 345, dtype: float64
